In [1]:
!pip install spotipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 5.7 MB/s eta 0:00:00


In [2]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth


client_id = '5b0dffb8f4d04048a43b57b22472b988'
client_secret = '02ec75f99deb496f9c76bc9608406e12'
redirect_uri = 'http://localhost:8888/callback'

sp = spotipy.Spotify(auth_manager= SpotifyOAuth(
    client_id= client_id,
    client_secret= client_secret,
    redirect_uri= redirect_uri,
    scope= 'user-top-read',
    open_browser= False
))

In [4]:
listening_history = sp.current_user_top_tracks(limit=50, offset=0, time_range='medium_term')
artist_history = sp.current_user_top_artists(limit=50, offset=0, time_range='medium_term')

Go to the following URL: https://accounts.spotify.com/authorize?client_id=5b0dffb8f4d04048a43b57b22472b988&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A8888%2Fcallback&scope=user-top-read
Enter the URL you were redirected to: http://localhost:8888/callback?code=AQDZQjnpJw8_po8lLmctLJQ1ue3Yp-r8Q3ZO1qoxwP6Bx59-uOY46i7nNgLLJa8TpFYvRHNeNk6alnrm2rBovh3Yv76vzVMwm1iWmWjmFWI0Wugfkyk94lDDhOSeSMrAG42z68ERg4qzSi9iXlloqhQgBBNNU8Uw4sH-O1jfVpTZQj7D_YhmRP-1KAVfdMYolw


In [5]:
import pandas as pd

In [6]:
for key in listening_history.keys():
    print(key)

print('\n')
for feature in listening_history['items'][0].keys():
    print(feature, type(feature))


items
total
limit
offset
href
next
previous


album <class 'str'>
artists <class 'str'>
available_markets <class 'str'>
disc_number <class 'str'>
duration_ms <class 'str'>
explicit <class 'str'>
external_ids <class 'str'>
external_urls <class 'str'>
href <class 'str'>
id <class 'str'>
is_local <class 'str'>
is_playable <class 'str'>
name <class 'str'>
popularity <class 'str'>
preview_url <class 'str'>
track_number <class 'str'>
type <class 'str'>
uri <class 'str'>


In [7]:
for key in artist_history.keys():
    print(key)

print('\n')
for feature in artist_history['items'][0].keys():
    print(feature, type(feature))

items
total
limit
offset
href
next
previous


external_urls <class 'str'>
followers <class 'str'>
genres <class 'str'>
href <class 'str'>
id <class 'str'>
images <class 'str'>
name <class 'str'>
popularity <class 'str'>
type <class 'str'>
uri <class 'str'>


In [8]:
pd.json_normalize(artist_history)

,items,total,limit,offset,href,next,previous
0,[{'external_urls': {'spotify': 'https://open.s...,204,50,0,https://api.spotify.com/v1/me/top/artists?offs...,https://api.spotify.com/v1/me/top/artists?offs...,None


In [9]:
df_artists = pd.json_normalize(artist_history['items'])
artist_features = ['genres', 'id', 'name', 'popularity', 'followers.total']
df_artists = df_artists[artist_features]
df_artists = df_artists.explode(column= 'genres').reset_index(drop=True)
df_artists

,genres,id,name,popularity,followers.total
0,cloud rap,3hGJ4nHdF99Vs0gQdXz5Nw,smokedope2016,55,71264
1,electroclash,6TsAG8Ve1icEC8ydeHm3C8,Snow Strippers,66,196706
2,witch house,6TsAG8Ve1icEC8ydeHm3C8,Snow Strippers,66,196706
3,lo-fi house,6G01WYFYF91rjG5LtwMhY4,1tbsp,46,32378
4,midwest emo,2CnhqfjUG0qzsru0SMuhrk,Title Fight,63,514218
...,...,...,...,...,...
112,hard house,5ye3YheRaIaodo3nid7Wxz,Upper90,40,7930
113,eurodance,5ye3YheRaIaodo3nid7Wxz,Upper90,40,7930
114,trance,5ye3YheRaIaodo3nid7Wxz,Upper90,40,7930
115,cloud rap,5tQMB0cuNXdCtzovGt55uD,LUCKI,77,1450295


In [10]:
while artist_history['next'] != None:
  artist_history = sp.next(artist_history)
  temp_artists = pd.json_normalize(artist_history['items'])
  temp_artists = temp_artists.explode(column= 'genres').reset_index(drop=True)
  temp_artists = temp_artists[artist_features]
  df_artists = pd.concat([df_artists, temp_artists], ignore_index=True)

df_artists

,genres,id,name,popularity,followers.total
0,cloud rap,3hGJ4nHdF99Vs0gQdXz5Nw,smokedope2016,55,71264
1,electroclash,6TsAG8Ve1icEC8ydeHm3C8,Snow Strippers,66,196706
2,witch house,6TsAG8Ve1icEC8ydeHm3C8,Snow Strippers,66,196706
3,lo-fi house,6G01WYFYF91rjG5LtwMhY4,1tbsp,46,32378
4,midwest emo,2CnhqfjUG0qzsru0SMuhrk,Title Fight,63,514218
...,...,...,...,...,...
396,hip hop,5me0Irg2ANcsgc93uaYrpb,The Notorious B.I.G.,76,11524392
397,east coast hip hop,7BMccF0hQFBpP6417k1OtQ,Action Bronson,61,851326
398,jazz rap,04aAAEbkuy8tC8xmaE0CcQ,Nitsua,41,16177
399,NaN,1ybINI1qPiFbwDXamRtwxD,Smino,66,1005430


In [11]:
df_artists.rename(columns= {'genres': 'subgenre'}, inplace= True)
df_artists

,subgenre,id,name,popularity,followers.total
0,cloud rap,3hGJ4nHdF99Vs0gQdXz5Nw,smokedope2016,55,71264
1,electroclash,6TsAG8Ve1icEC8ydeHm3C8,Snow Strippers,66,196706
2,witch house,6TsAG8Ve1icEC8ydeHm3C8,Snow Strippers,66,196706
3,lo-fi house,6G01WYFYF91rjG5LtwMhY4,1tbsp,46,32378
4,midwest emo,2CnhqfjUG0qzsru0SMuhrk,Title Fight,63,514218
...,...,...,...,...,...
396,hip hop,5me0Irg2ANcsgc93uaYrpb,The Notorious B.I.G.,76,11524392
397,east coast hip hop,7BMccF0hQFBpP6417k1OtQ,Action Bronson,61,851326
398,jazz rap,04aAAEbkuy8tC8xmaE0CcQ,Nitsua,41,16177
399,NaN,1ybINI1qPiFbwDXamRtwxD,Smino,66,1005430


In [ ]:
for genre in df_artists['subgenre'].unique():
  print(genre)

cloud rap
electroclash
witch house
darkwave
post-punk
cold wave
lo-fi house
midwest emo
shoegaze
post-hardcore
melodic hardcore
pop punk
eurotrance
hard house
eurodance
techno
dark trap
underground hip hop
memphis rap
emo rap
hyperpop
phonk
hard techno
acid techno
tekno
jazz rap
stutter house
house
nan
trance
nu jazz
reggae
roots reggae
melodic rap
dubstep
edm
electro
electronic
punk
rock
emo
skate punk
breakcore
jungle
speedcore
breakbeat
drum and bass
reggae rock
ska punk
ska
bedroom pop
indie pop
rage rap
nu metal
alternative metal
rap metal
hip hop
rap
punk rap
melodic house
progressive house
deep house
french house
riddim
deathstep
bass music
rave
dub
psytrance
dream pop
britpop
slowcore
experimental hip hop
southern hip hop
crunk
gangster rap
post-grunge
indie
psychedelic pop
alternative hip hop
idm
ambient
electronica
trap
tech house
classic rock
soft rock
melodic techno
soundtrack
reggaeton
west coast hip hop
drill
future house
merengue
música tropical
east coast hip hop
boom b

In [ ]:
rap_idx = [78, 114, 164, 200, 208, 210, 211, 214, 224, 225, 257, 231, 232, 245, 246, 254, 255, 256, 275, 278, 281, 343, 326, 322, 339, 354, 358, 362,388, 393, 392, 395, 397, 399]
indie_idx = [37, 48, 55, 346, 341, 288, 267, 192, 213, 183, 176, 377, 379, 154, 100, 152, 212, 350, 345, 291, 270]
electronic_idx = [92, 172, 325, 321, 375, 329]
house_idx = [295]
ambient_idx = [292]
jazz_idx = [178]
soul_idx = [287, 85, 344, 348, 290]

na_mapping = {
    'rap': rap_idx,
    'indie': indie_idx,
    'electronic': electronic_idx,
    'house': house_idx,
    'ambient': ambient_idx,
    'nu jazz': jazz_idx,
    'soul': soul_idx

    }

for q in na_mapping.keys():
  df_artists.loc[na_mapping[q], 'subgenre'] = df_artists.loc[na_mapping[q], 'subgenre'].fillna(q)

print(df_artists['subgenre'].isna().sum())
df_artists[df_artists['subgenre'].isna()]

20


,subgenre,id,name,popularity,followers.total
35,NaN,0U7iI0Dk4Ojvi17nZboNO4,Vacations,73,1265228
50,NaN,3XxNRirzbjfLdDli06zMaB,Eyedress,75,2393415
115,NaN,1anyVhU62p31KFi8MEzkbf,Chance the Rapper,74,6026729
191,NaN,6bJCrLZcvsBMzve04BmgwS,Golden Vessel,45,60294
199,NaN,51Pn4gGnZuyOMDfIT1Zy3N,The Underachievers,46,380850
209,NaN,4LLpKhyESsyAXpc4laK94U,Mac Miller,83,11281494
258,NaN,1Nd1g8fezKoJ6DGauWbZIm,Beast Coast,41,120742
271,NaN,4o6dCIkmONTJL0ByDDUNYd,Mind’s Eye,47,56662
279,NaN,7e10JUMF7MJmmwYpnTSMI5,Dave East,56,702045
282,NaN,4xRYI6VqpkE3UwrDrAZL8L,Logic,73,6110012


In [ ]:
df_artists.rename(columns= {'id': 'artist_id', 'name': 'artist', 'followers.total': 'followers'}, inplace= True)
df_artists

,subgenre,artist_id,artist,popularity,followers
0,electroclash,6TsAG8Ve1icEC8ydeHm3C8,Snow Strippers,68,188890
1,witch house,6TsAG8Ve1icEC8ydeHm3C8,Snow Strippers,68,188890
2,cloud rap,3hGJ4nHdF99Vs0gQdXz5Nw,smokedope2016,55,68128
3,darkwave,62s5q1tm3UAcRnbGY9CCI9,Provoker,49,62967
4,post-punk,62s5q1tm3UAcRnbGY9CCI9,Provoker,49,62967
...,...,...,...,...,...
396,hip hop,5me0Irg2ANcsgc93uaYrpb,The Notorious B.I.G.,77,11450478
397,rap,6xS5PpBWaVYraexEkEjjXv,Maxo Kream,59,472461
398,jazz rap,04aAAEbkuy8tC8xmaE0CcQ,Nitsua,43,16030
399,rap,1ybINI1qPiFbwDXamRtwxD,Smino,68,1002295


In [ ]:
genre_mapping = {
    'Hip Hop':['east coast hip hop', 'boom bap','g-funk', 'hardcore hip hop', 'west coast hip hop', 'gangster rap', 'crunk', 'southern hip hop'],
    'Rap': ['indie rap','experimental hip hop','trap', 'drill', 'horrorcore' ,'underground hip hop', 'memphis rap', 'cloud rap', 'emo rap', 'melodic rap', 'rap metal', 'rage rap', 'hip hop', 'rap', 'dark trap'] ,
    'House': ['tech house','future house', 'afro house','french house', 'stutter house', 'house', 'deep house', 'hard house', 'melodic house', 'lo-fi house', 'progressive house', 'electro house', 'piano house'],
    'Dance': ['psytrance','uk funky', 'trance', 'phonk', 'eurodance', 'edm', 'eurotrance', 'italo dance', 'happy hardcore', 'brazilian funk', 'kuduro', 'alternative dance'],
    'Electronic': ['indietronica', 'electroclash', 'witch house', 'riddim', 'bass music', 'new rave', 'rave', 'dub', 'dubstep', 'speedcore', 'drum and bass', 'breakcore', 'hyperpop', 'jungle', 'electronic', 'deathstep', 'breakbeat', 'electro', 'uk garage', 'electronica'],
    'Ambient': ['ambient', 'idm'],
    'Techno': ['hard techno', 'acid techno', 'tekno', 'techno', 'melodic techno'],
    'Alternative Rock': ['metalcore', 'post-grunge', 'slowcore', 'shoegaze', 'midwest emo', 'darkwave', 'coldwave', 'emo', 'alternative metal', 'post-hardcore', 'melodic hardcore', 'nu metal', 'cold wave'],
    'Jazz': ['nu jazz', 'jazz funk', 'jazz fusion', 'jazz rap'],
    'Latin': ['latin', 'urbano latino', 'trap latino', 'merengue', 'música tropical', 'reggaeton'],
    'Soul': ['retro soul', 'soul'],
    'Video Game': ['soundtrack'],
    'Punk': ['post-punk', 'skate punk', 'punk', 'pop punk', 'punk rap'],
    'Rock': ['rock', 'reggae rock', 'electronic rock', 'hard rock', 'acid rock', 'psychedelic rock', 'blues rock', 'soft rock', 'classic rock'],
    'Reggae': ['ska', 'ska punk', 'reggae', 'roots reggae'],
    'Indie': [None, 'indie pop', 'bedroom pop', 'psychedelic pop', 'indie', 'britpop', 'dream pop']
     }
sum = 0
for genre in genre_mapping.keys():
  sum = sum + len(genre_mapping[genre])
print(sum)

126


In [ ]:
def map_genre(subgenres):
    return next((genre for genre, subgenre_list in genre_mapping.items() if any(subgenre in subgenre_list for subgenre in subgenres)), None)

df_artists['genre'] = df_artists['subgenre'].apply(lambda x: map_genre([x]) if isinstance(x, str) else None)

In [ ]:
for artist, group in df_artists.groupby('artist'):
  mode_genre = group['genre'].mode()[0]
  df_artists.loc[group.index, 'genre'] = mode_genre

df_artists.drop_duplicates(subset= ['artist_id', 'genre'], inplace= True)
df_artists.reset_index(drop= True, inplace= True)
df_artists

,subgenre,artist_id,artist,popularity,followers,genre
0,electroclash,6TsAG8Ve1icEC8ydeHm3C8,Snow Strippers,68,188890,Electronic
1,cloud rap,3hGJ4nHdF99Vs0gQdXz5Nw,smokedope2016,55,68128,Rap
2,darkwave,62s5q1tm3UAcRnbGY9CCI9,Provoker,49,62967,Alternative Rock
3,lo-fi house,6G01WYFYF91rjG5LtwMhY4,1tbsp,47,31722,House
4,midwest emo,2CnhqfjUG0qzsru0SMuhrk,Title Fight,65,508715,Alternative Rock
...,...,...,...,...,...,...
201,gangster rap,5me0Irg2ANcsgc93uaYrpb,The Notorious B.I.G.,77,11450478,Hip Hop
202,rap,6xS5PpBWaVYraexEkEjjXv,Maxo Kream,59,472461,Rap
203,jazz rap,04aAAEbkuy8tC8xmaE0CcQ,Nitsua,43,16030,Jazz
204,rap,1ybINI1qPiFbwDXamRtwxD,Smino,68,1002295,Rap


In [ ]:
df_tracks = pd.json_normalize(listening_history['items'])
features = ['artists', 'duration_ms', 'explicit', 'id', 'name', 'popularity', 'album.name', 'album.release_date', 'album.total_tracks']
df_tracks = df_tracks[features]
df_tracks = df_tracks.explode(column= 'artists').reset_index(drop=True)
df_tracks['artist'] = ''
df_tracks['artist_id'] = ''
for idx,row in df_tracks['artists'].items():
  df_tracks.loc[idx, 'artist']= row['name']
  df_tracks.loc[idx, 'artist_id'] = row['id']

df_tracks

,artists,duration_ms,explicit,id,name,popularity,album.name,album.release_date,album.total_tracks,artist,artist_id
0,{'external_urls': {'spotify': 'https://open.sp...,218500,False,3iRoXGFrF8O0jTul6LAmus,Under Your Spell,78,April Mixtape 3,2023-05-05,16,Snow Strippers,6TsAG8Ve1icEC8ydeHm3C8
1,{'external_urls': {'spotify': 'https://open.sp...,201034,False,3C4qukRl7kX1wdevwbOudf,Vehicle Dissolve,33,Body Jumper,2021-08-13,13,Provoker,62s5q1tm3UAcRnbGY9CCI9
2,{'external_urls': {'spotify': 'https://open.sp...,63817,True,3iBtk9V7k32rkvE9bSKJWc,gonmakit,46,gonmakit,2024-05-10,1,Brennan Jones,4PaHlM2H2hC3VQV4BfLo4B
3,{'external_urls': {'spotify': 'https://open.sp...,151273,False,4qeIlzP0Z1JFXh0AckDcX8,No Nein,54,No Nein,2023-02-14,1,1tbsp,6G01WYFYF91rjG5LtwMhY4
4,{'external_urls': {'spotify': 'https://open.sp...,151273,False,4qeIlzP0Z1JFXh0AckDcX8,No Nein,54,No Nein,2023-02-14,1,Mietze Conte,3ETvRzzQK7ppB28GcAE8rN
...,...,...,...,...,...,...,...,...,...,...,...
66,{'external_urls': {'spotify': 'https://open.sp...,208457,False,0Tcy9RRWvcnqepvmVcqv05,Black Out Days - Subtronics Remix,58,Black Out Days (Subtronics Remix),2023-06-16,1,Subtronics,3NJ94iuAmmMjbszODYT6pO
67,{'external_urls': {'spotify': 'https://open.sp...,251586,False,1gMT8boiNdGWiYy7HQHyZk,XENA,47,Quest For Fire,2023-02-17,15,Skrillex,5he5w2lnU9x7JFhnwcekXX
68,{'external_urls': {'spotify': 'https://open.sp...,251586,False,1gMT8boiNdGWiYy7HQHyZk,XENA,47,Quest For Fire,2023-02-17,15,Nai Barghouti,78XHgIjAv0tqb9hVRUsifg
69,{'external_urls': {'spotify': 'https://open.sp...,255840,False,7BfW1eoDh27W69nxsmRicb,Buffalo Soldier,70,Confrontation,1983-05-23,11,Bob Marley & The Wailers,2QsynagSdAqZj3U9HgDzjD


In [ ]:
while listening_history['next'] != None:
  listening_history = sp.next(listening_history)
  temp_tracks = pd.json_normalize(listening_history['items'])
  temp_tracks = temp_tracks[features]
  temp_tracks = temp_tracks.explode(column= 'artists').reset_index(drop=True)
  temp_tracks['artist'] = ''
  temp_tracks['artist_id'] = ''
  for idx,row in temp_tracks['artists'].items():
    temp_tracks.loc[idx, 'artist'] = row['name']
    temp_tracks.loc[idx, 'artist_id'] = row['id']
  temp_tracks.drop(columns= 'artists', inplace=True)
  df_tracks = pd.concat([df_tracks, temp_tracks], ignore_index=True)


df_tracks.drop(columns= 'artists', inplace=True)
df_tracks

,duration_ms,explicit,id,name,popularity,album.name,album.release_date,album.total_tracks,artist,artist_id
0,218500,False,3iRoXGFrF8O0jTul6LAmus,Under Your Spell,78,April Mixtape 3,2023-05-05,16,Snow Strippers,6TsAG8Ve1icEC8ydeHm3C8
1,201034,False,3C4qukRl7kX1wdevwbOudf,Vehicle Dissolve,33,Body Jumper,2021-08-13,13,Provoker,62s5q1tm3UAcRnbGY9CCI9
2,63817,True,3iBtk9V7k32rkvE9bSKJWc,gonmakit,46,gonmakit,2024-05-10,1,Brennan Jones,4PaHlM2H2hC3VQV4BfLo4B
3,151273,False,4qeIlzP0Z1JFXh0AckDcX8,No Nein,54,No Nein,2023-02-14,1,1tbsp,6G01WYFYF91rjG5LtwMhY4
4,151273,False,4qeIlzP0Z1JFXh0AckDcX8,No Nein,54,No Nein,2023-02-14,1,Mietze Conte,3ETvRzzQK7ppB28GcAE8rN
...,...,...,...,...,...,...,...,...,...,...
3952,341967,False,7k0mvImLkHXeiaAYDQDnyB,"In Europe, Kinda Sad",14,kanashī,2021-03-17,4,1tbsp,6G01WYFYF91rjG5LtwMhY4
3953,108000,False,7rdd9cSRRbO5wF8YFvKVIj,paranoid_2_outside_the_party,40,Im_my_worst_enemy,2021-02-01,9,Harto Falión,0wAEYYaHOqYJhHaSvpZUwP
3954,108000,False,7rdd9cSRRbO5wF8YFvKVIj,paranoid_2_outside_the_party,40,Im_my_worst_enemy,2021-02-01,9,evilgiane,4bbAeMRdMbfCLVSlpryAVV
3955,112243,True,7tHXd3CsT7VGZAJ5Jkqt61,nothing like uuu,32,nothing like uuu,2024-04-29,1,Nettspend,2jl4qd6UbzeCmImT4nWbtA


In [ ]:
len(df_tracks['artist'].unique())

1604

In [ ]:
df_tracks.rename(columns= {'id': 'track_id', 'name': 'track', 'album.name': 'album', 'album.release_date': 'album_release_date', 'album.total_tracks': 'album_total_tracks'}, inplace=True)
df_tracks

,duration_ms,explicit,track_id,track,popularity,album,album_release_date,album_total_tracks,artist,artist_id
0,218500,False,3iRoXGFrF8O0jTul6LAmus,Under Your Spell,78,April Mixtape 3,2023-05-05,16,Snow Strippers,6TsAG8Ve1icEC8ydeHm3C8
1,201034,False,3C4qukRl7kX1wdevwbOudf,Vehicle Dissolve,33,Body Jumper,2021-08-13,13,Provoker,62s5q1tm3UAcRnbGY9CCI9
2,63817,True,3iBtk9V7k32rkvE9bSKJWc,gonmakit,46,gonmakit,2024-05-10,1,Brennan Jones,4PaHlM2H2hC3VQV4BfLo4B
3,151273,False,4qeIlzP0Z1JFXh0AckDcX8,No Nein,54,No Nein,2023-02-14,1,1tbsp,6G01WYFYF91rjG5LtwMhY4
4,151273,False,4qeIlzP0Z1JFXh0AckDcX8,No Nein,54,No Nein,2023-02-14,1,Mietze Conte,3ETvRzzQK7ppB28GcAE8rN
...,...,...,...,...,...,...,...,...,...,...
3952,341967,False,7k0mvImLkHXeiaAYDQDnyB,"In Europe, Kinda Sad",14,kanashī,2021-03-17,4,1tbsp,6G01WYFYF91rjG5LtwMhY4
3953,108000,False,7rdd9cSRRbO5wF8YFvKVIj,paranoid_2_outside_the_party,40,Im_my_worst_enemy,2021-02-01,9,Harto Falión,0wAEYYaHOqYJhHaSvpZUwP
3954,108000,False,7rdd9cSRRbO5wF8YFvKVIj,paranoid_2_outside_the_party,40,Im_my_worst_enemy,2021-02-01,9,evilgiane,4bbAeMRdMbfCLVSlpryAVV
3955,112243,True,7tHXd3CsT7VGZAJ5Jkqt61,nothing like uuu,32,nothing like uuu,2024-04-29,1,Nettspend,2jl4qd6UbzeCmImT4nWbtA


In [ ]:
df_tracks['album_release_date'] = df_tracks['album_release_date'].apply(lambda x: f'{x}-01-01' if len(x) == 4 else x)
df_tracks['album_release_date'] = df_tracks['album_release_date'].apply(lambda x: f'{x}-01' if len(x) == 7 else x)
df_tracks['album_release_date'] = pd.to_datetime(df_tracks['album_release_date'])
df_tracks

,duration_ms,explicit,track_id,track,popularity,album,album_release_date,album_total_tracks,artist,artist_id
0,218500,False,3iRoXGFrF8O0jTul6LAmus,Under Your Spell,78,April Mixtape 3,2023-05-05,16,Snow Strippers,6TsAG8Ve1icEC8ydeHm3C8
1,201034,False,3C4qukRl7kX1wdevwbOudf,Vehicle Dissolve,33,Body Jumper,2021-08-13,13,Provoker,62s5q1tm3UAcRnbGY9CCI9
2,63817,True,3iBtk9V7k32rkvE9bSKJWc,gonmakit,46,gonmakit,2024-05-10,1,Brennan Jones,4PaHlM2H2hC3VQV4BfLo4B
3,151273,False,4qeIlzP0Z1JFXh0AckDcX8,No Nein,54,No Nein,2023-02-14,1,1tbsp,6G01WYFYF91rjG5LtwMhY4
4,151273,False,4qeIlzP0Z1JFXh0AckDcX8,No Nein,54,No Nein,2023-02-14,1,Mietze Conte,3ETvRzzQK7ppB28GcAE8rN
...,...,...,...,...,...,...,...,...,...,...
3952,341967,False,7k0mvImLkHXeiaAYDQDnyB,"In Europe, Kinda Sad",14,kanashī,2021-03-17,4,1tbsp,6G01WYFYF91rjG5LtwMhY4
3953,108000,False,7rdd9cSRRbO5wF8YFvKVIj,paranoid_2_outside_the_party,40,Im_my_worst_enemy,2021-02-01,9,Harto Falión,0wAEYYaHOqYJhHaSvpZUwP
3954,108000,False,7rdd9cSRRbO5wF8YFvKVIj,paranoid_2_outside_the_party,40,Im_my_worst_enemy,2021-02-01,9,evilgiane,4bbAeMRdMbfCLVSlpryAVV
3955,112243,True,7tHXd3CsT7VGZAJ5Jkqt61,nothing like uuu,32,nothing like uuu,2024-04-29,1,Nettspend,2jl4qd6UbzeCmImT4nWbtA


In [ ]:
df_features = pd.DataFrame(columns= ['track_id', 'track', 'artist', 'artist_id'])

for track, group in df_tracks.groupby('track'):
  if len(group['artist']) > 1:
    for artist in group['artist'][1:]:
      df_features.loc[len(df_features)] = [group.loc[group['artist'] == artist, 'track_id'].values[0], track, artist, group.loc[group['artist'] == artist, 'artist_id'].values[0]]

df_features

,track_id,track,artist,artist_id
0,0tgAhrvNPhSyMZR0wWbv6Z,#1 Freak (feat. Ty Dolla $ign),Ty Dolla $ign,7c0XG5cIJTrrAgEC3ULPiq
1,5fupW8u36XYYlA9IEQI4LJ,#TWITCHING,DJmegan23,56zdYcwtmdYq3WlBYQeHoA
2,1TkWsnheqK0gBtvh07BtNX,$ave Dat Money (feat. Fetty Wap & Rich Homie Q...,Fetty Wap,6PXS4YHDkKvl1wkIl4V8DL
3,1TkWsnheqK0gBtvh07BtNX,$ave Dat Money (feat. Fetty Wap & Rich Homie Q...,Rich Homie Quan,5lHRUCqkQZCIWeX7xG4sYT
4,5gmkw2031VVIboYqPIjSWB,1 Scale (feat. G Herbo),G Herbo,5QdEbQJ3ylBnc3gsIASAT5
...,...,...,...,...
1328,4S05mkyTtAiWy5l4umch0X,where will i be,Emmylou Harris,5s6TJEuHTr9GR894wc6VfP
1329,59J5nzL1KniFHnU120dQzt,wokeuplikethis*,Lil Uzi Vert,4O15NlyKLIASxsJ0PrXPfz
1330,11qCHQPkZU9L3vvIYQbEGn,Осенний кот,Стас Экстаз,4ciTih43wrAFNsGK9IB2Qt
1331,3oID9FwJyDT5CUfFlxZClc,ただdance in the dark - Mixed,Peterparker69,1tgZVOBOut0nS8CJ5SIBh5


In [ ]:
df_features.drop_duplicates(subset= ['track', 'artist'], inplace= True)
df_features.reset_index(drop= True, inplace= True)
df_features

,track_id,track,artist,artist_id
0,0tgAhrvNPhSyMZR0wWbv6Z,#1 Freak (feat. Ty Dolla $ign),Ty Dolla $ign,7c0XG5cIJTrrAgEC3ULPiq
1,5fupW8u36XYYlA9IEQI4LJ,#TWITCHING,DJmegan23,56zdYcwtmdYq3WlBYQeHoA
2,1TkWsnheqK0gBtvh07BtNX,$ave Dat Money (feat. Fetty Wap & Rich Homie Q...,Fetty Wap,6PXS4YHDkKvl1wkIl4V8DL
3,1TkWsnheqK0gBtvh07BtNX,$ave Dat Money (feat. Fetty Wap & Rich Homie Q...,Rich Homie Quan,5lHRUCqkQZCIWeX7xG4sYT
4,5gmkw2031VVIboYqPIjSWB,1 Scale (feat. G Herbo),G Herbo,5QdEbQJ3ylBnc3gsIASAT5
...,...,...,...,...
1285,4S05mkyTtAiWy5l4umch0X,where will i be,Emmylou Harris,5s6TJEuHTr9GR894wc6VfP
1286,59J5nzL1KniFHnU120dQzt,wokeuplikethis*,Lil Uzi Vert,4O15NlyKLIASxsJ0PrXPfz
1287,11qCHQPkZU9L3vvIYQbEGn,Осенний кот,Стас Экстаз,4ciTih43wrAFNsGK9IB2Qt
1288,3oID9FwJyDT5CUfFlxZClc,ただdance in the dark - Mixed,Peterparker69,1tgZVOBOut0nS8CJ5SIBh5


In [ ]:
df_tracks = df_tracks.drop_duplicates(subset= ['track_id'])
df_tracks.reset_index(drop= True, inplace= True)
df_tracks

,duration_ms,explicit,track_id,track,popularity,album,album_release_date,album_total_tracks,artist,artist_id
0,218500,False,3iRoXGFrF8O0jTul6LAmus,Under Your Spell,78,April Mixtape 3,2023-05-05,16,Snow Strippers,6TsAG8Ve1icEC8ydeHm3C8
1,201034,False,3C4qukRl7kX1wdevwbOudf,Vehicle Dissolve,33,Body Jumper,2021-08-13,13,Provoker,62s5q1tm3UAcRnbGY9CCI9
2,151273,False,4qeIlzP0Z1JFXh0AckDcX8,No Nein,54,No Nein,2023-02-14,1,1tbsp,6G01WYFYF91rjG5LtwMhY4
3,210109,False,0ej3RoZrDPhwQ7A6T8UoIS,Sleeves Touch My Elbows,41,Mosquito Love,2023-08-17,8,1tbsp,6G01WYFYF91rjG5LtwMhY4
4,176306,False,18K49IqrtMHXpXujaKAzsc,Hypernight,41,Hyperview,2015-02-03,10,Title Fight,2CnhqfjUG0qzsru0SMuhrk
...,...,...,...,...,...,...,...,...,...,...
2721,231586,False,6zbCsfnPxrzFIaQf6DV3a5,Give It Up,31,Hovvdy,2024-04-26,19,Hovvdy,59RNNqeEfkq3X5pfOQxZ3C
2722,341967,False,7k0mvImLkHXeiaAYDQDnyB,"In Europe, Kinda Sad",14,kanashī,2021-03-17,4,1tbsp,6G01WYFYF91rjG5LtwMhY4
2723,108000,False,7rdd9cSRRbO5wF8YFvKVIj,paranoid_2_outside_the_party,40,Im_my_worst_enemy,2021-02-01,9,Harto Falión,0wAEYYaHOqYJhHaSvpZUwP
2724,112243,True,7tHXd3CsT7VGZAJ5Jkqt61,nothing like uuu,32,nothing like uuu,2024-04-29,1,Nettspend,2jl4qd6UbzeCmImT4nWbtA


In [ ]:
df_tracks.to_csv('spotify_top_tracks.csv')
df_artists.to_csv('spotify_top_artists.csv')


In [ ]:
df_features.to_csv('spotify_top_featured_artists.csv')